In [119]:
# Import necessary libraries
import base64
import json
import sys 
import os
import fitz
import requests
import re
from fitz import Document, Page, Rect

## Load page 2 and crop to left page (page 4) 

In [120]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(2)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page_left = new_doc.new_page(-1, width=r1.width, height=r1.height)
new_page_left.show_pdf_page(
    new_page_left.rect,
    src,
    page.number,
    clip=r1,
)

80

### 1. Option to use get_text()

In [121]:
text_bytes_left = new_page_left.get_text().encode("utf8") # get plain text (is in UTF-8)
text_left = text_bytes_left.decode('utf-8')
print(text_left)

574
4
einsicht
WIENER WINKEL
wos is los …
… beim Augustin
Pseudo-Christo, 1010 Wien 
Foto: Nina Strasser
D
as deutschsprachige Straßenzeitungstreffen ende 
april in Hamburg stand im Zeichen des Zusam-
menhalts. 21 Straßenzeitungen aus Österreich, 
Deutschland und der Schweiz sind der einladung der 
Kolleg:innen von Hinz&Kunzt gefolgt, haben sich ausge-
tauscht und vor allem gegenseitig bestärkt. Denn  Stärke 
brauchen wir jetzt, um unsere arbeit weiterzuführen. 
Die Teuerungen bringen mehr armut – wir sind mehr 
gefordert. Um diese zusätzliche Unterstützungsarbeit 
für  Betroffene leisten zu können, ohne 
uns dabei selbst auszubeuten, haben 
wir uns bei der Konferenz ausgiebig 
dem Thema «Fundraising» gewidmet. 
nicht nur die entwicklung der Print-
medien, auch Corona hat uns allen 
einbußen bei den  verkaufszahlen 
 beschert. Wir wollen aber weiterhin 
 Unterstützung in sozialrechtlichen Fra-
gen, Begleitung bei medizinischen notfällen, Zur-verfü-
gung-Stellen von Speis & Trank, Recht

### 2. Option to print out each line with meta information

In [122]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = new_page_left.get_text("dict", flags=11)["blocks"]
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            print("")
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            print("Text: '%s'" % s["text"])  # simple print of text
            print(font_properties)


Text: '574'
Font: 'MyriadPro-BlackCond' (serifed, proportional, bold), size 13.9298, color #f3664e

Text: '4'
Font: 'MyriadPro-Bold' (serifed, proportional, bold), size 24, color #000000

Text: 'einsicht'
Font: 'MyriadPro-Regular' (serifed, proportional), size 14.3543, color #f3664e

Text: 'WIENER WINKEL'
Font: 'MyriadPro-SemiCn' (serifed, proportional), size 24.2388, color #000000

Text: 'wos is los …'
Font: 'MyriadPro-Semibold' (serifed, proportional, bold), size 14.9238, color #000000

Text: '… beim Augustin'
Font: 'MyriadPro-Semibold' (serifed, proportional, bold), size 14.9238, color #000000

Text: 'Pseudo-Christo, 1010 Wien '
Font: 'MyriadPro-BoldSemiCn' (serifed, proportional, bold), size 7.95019, color #000000

Text: 'Foto: Nina Strasser'
Font: 'MyriadPro-BoldSemiCn' (serifed, proportional, bold), size 7.95019, color #000000

Text: 'D'
Font: 'Sentinel-Book' (serifed, proportional), size 37.8081, color #f3664e

Text: 'as deutschsprachige Straßenzeitungstreffen ende '
Font: 'Myr

### 3. Option to get text from rectangle using get_textbox(rect)

In [123]:
rect = fitz.Rect(0, 400, 300, 800)
print(new_page_left.get_textbox(rect))

wos is los …
… beim Augustin
D
as deutschsprachige Straßenzeitungstreffen ende 
april in Hamburg stand im Zeichen des Zusam-
menhalts. 21 Straßenzeitungen aus Österreich, 
Deutschland und der Schweiz sind der einladung der 
Kolleg:innen von Hinz&Kunzt gefolgt, haben sich ausge-
tauscht und vor allem gegenseitig bestärkt. Denn  Stärke 
brauchen wir jetzt, um unsere arbeit weiterzuführen. 
Die Teuerungen bringen mehr armut – wir sind mehr 
gefordert. Um diese zusätzliche Unterstützungsarbeit 
für  Betroffene leisten zu können, ohne 
uns dabei selbst auszubeuten, haben 
wir uns bei der Konferenz ausgiebig 
dem Thema «Fundraising» gewidmet. 
nicht nur die entwicklung der Print-
medien, auch Corona hat uns allen 
einbußen bei den  verkaufszahlen 
 beschert. Wir wollen aber weiterhin 
 Unterstützung in sozialrechtlichen Fra-
gen, Begleitung bei medizinischen notfällen, Zur-verfü-
gung-Stellen von Speis & Trank, Rechtsberatung und Teil-
nahme an Kulturprojekten anbieten. 
Wie können wir all d

## Load page 2 and crop to right page (page 5) 

In [124]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(2)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page_right = new_doc.new_page(-1, width=r2.width, height=r2.height)
new_page_right.show_pdf_page(
    new_page_right.rect,
    src,
    page.number,
    clip=r2,
)

80

### 1. Option to use get_text()

In [125]:
text = new_page_right.get_text()
print(text)     

574
5
das wahre leben 
F
ür eine gute Kinder- und Jugendhilfe sind gleiche Standards 
vom Neusiedler- bis zum Bodensee Voraussetzung.  Gerade 
Jugendliche mit schwieriger Lebensgeschichte brauchen 
 Begleitung und Betreuung über das 18. Lebensjahr hinaus. In 
Österreich macht es einen Unterschied, wo ein Kind lebt. Die Hilfen 
unterscheiden sich von Bundesland zu Bundesland.
Jugendliche, die niemanden haben, der:die für sie sorgt, müssen 
dieselben Chancen erhalten wie jene Kinder, die in Familien auf-
wachsen dürfen. Deshalb wäre es vernünftig, dass 
junge Menschen auch über den 18. Geburtstag hi-
naus einen Rechtsanspruch auf  Kinder- und Ju-
gendhilfe bekommen. Auch in  einer Familie endet 
die Sorge und Unterstützung nicht einfach mit dem 
18. Geburtstag. Und hier geht es um Jugendliche 
mit schwierigsten Lebensgeschichten. Diese Be-
gleitung wirkt stark präventiv und beugt Abstür-
zen vor, wie wir aus anderen europäischen Län-
dern wissen. Im Schnitt ziehen junge Menschen hierzula

In [126]:
words = new_page_right.get_text("words") # get plain text (is in UTF-8)
print(words)

[(26.389549255371094, 38.286529541015625, 42.34954833984375, 56.74810791015625, '574', 0, 0, 0), (587.6253662109375, 31.114707946777344, 600.9453735351562, 60.8507080078125, '5', 1, 0, 0), (472.0771484375, 42.076210021972656, 492.5679626464844, 58.4757080078125, 'das', 1, 1, 0), (495.58056640625, 42.076210021972656, 532.3419799804688, 58.4757080078125, 'wahre', 1, 1, 1), (535.3546142578125, 42.076210021972656, 568.9186401367188, 58.4757080078125, 'leben', 1, 1, 2), (33.75738525390625, 169.43218994140625, 66.89643859863281, 254.17079162597656, 'F', 2, 0, 0), (67.41845703125, 191.945556640625, 76.35286712646484, 205.0365447998047, 'ür', 3, 0, 0), (77.80322265625, 191.945556640625, 94.86323547363281, 205.0365447998047, 'eine', 3, 0, 1), (96.3135986328125, 191.945556640625, 113.58558654785156, 205.0365447998047, 'gute', 3, 0, 2), (115.03594970703125, 191.945556640625, 146.61790466308594, 205.0365447998047, 'Kinder-', 3, 0, 3), (148.0682373046875, 191.945556640625, 163.53846740722656, 205.0

### 2. Option to print out each line with meta information

In [127]:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = new_page_right.get_text("dict", flags=11)["blocks"]
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            print("")
            font_properties = "Font: '%s' (%s), size %g, color #%06x" % (
                s["font"],  # font name
                flags_decomposer(s["flags"]),  # readable font flags
                s["size"],  # font size
                s["color"],  # font color
            )
            print("Text: '%s'" % s["text"])  # simple print of text
            print(font_properties)


Text: '574'
Font: 'MyriadPro-BlackCond' (serifed, proportional, bold), size 13.9298, color #f3664e

Text: '5'
Font: 'MyriadPro-Bold' (serifed, proportional, bold), size 24, color #000000

Text: 'das wahre leben '
Font: 'MyriadPro-Regular' (serifed, proportional), size 14.3543, color #f3664e

Text: 'F'
Font: 'Sentinel-Book' (serifed, proportional), size 56.6946, color #f3664e

Text: 'ür eine gute Kinder- und Jugendhilfe sind gleiche Standards '
Font: 'Sentinel-Book' (serifed, proportional), size 9.39795, color #000000

Text: 'vom Neusiedler- bis zum Bodensee Voraussetzung.  Gerade '
Font: 'Sentinel-Book' (serifed, proportional), size 9.40452, color #000000

Text: 'Jugendliche mit schwieriger Lebensgeschichte brauchen '
Font: 'Sentinel-Book' (serifed, proportional), size 9.48183, color #000000

Text: ' Begleitung und Betreuung über das 18. Lebensjahr hinaus. In '
Font: 'Sentinel-Book' (serifed, proportional), size 9.37128, color #000000

Text: 'Österreich macht es einen Unterschied, wo 

### 3. Option to get text from rectangle using get_textbox(rect)

In [130]:
rect_right = fitz.Rect(0, 100, 600, 800)
print(new_page_right.get_textbox(rect_right))

F
ür eine gute Kinder- und Jugendhilfe sind gleiche Standards 
vom Neusiedler- bis zum Bodensee Voraussetzung.  Gerade 
Jugendliche mit schwieriger Lebensgeschichte brauchen 
 Begleitung und Betreuung über das 18. Lebensjahr hinaus. In 
Österreich macht es einen Unterschied, wo ein Kind lebt. Die Hilfen 
unterscheiden sich von Bundesland zu Bundesland.
Jugendliche, die niemanden haben, der:die für sie sorgt, müssen 
dieselben Chancen erhalten wie jene Kinder, die in Familien auf-
wachsen dürfen. Deshalb wäre es vernünftig, dass 
junge Menschen auch über den 18. Geburtstag hi-
naus einen Rechtsanspruch auf  Kinder- und Ju-
gendhilfe bekommen. Auch in  einer Familie endet 
die Sorge und Unterstützung nicht einfach mit dem 
18. Geburtstag. Und hier geht es um Jugendliche 
mit schwierigsten Lebensgeschichten. Diese Be-
gleitung wirkt stark präventiv und beugt Abstür-
zen vor, wie wir aus anderen europäischen Län-
dern wissen. Im Schnitt ziehen junge Menschen hierzulande mit 24 
Jahren von 